In [233]:
import pandas as pd
import numpy as np
from plotnine import *
from mizani.formatters import comma_format, percent_format
from tqdm.notebook import tqdm
from datetime import datetime
import requests
import json

%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


### Using the PubliBike API using `requests`

Lesson of the day: Read documentation before you start investing time into something that you would have learned does not actually work because of rate limits.

In [239]:
response = requests.get("https://api.publibike.ch/v1/public/partner/stations")
content = response.json()

<Response [200]>

In [244]:
stations = pd.json_normalize(content["stations"], sep="_").assign(
    timestamp=pd.to_datetime(datetime.now().strftime("%Y-%m-%d %H:%M"))
)
# get datetime col to front
stations = pd.concat([stations["timestamp"], stations.drop("timestamp", axis=1)], axis=1)

stations.head()

,timestamp,id,latitude,longitude,name,address,zip,city,vehicles,sponsors,is_virtual_station,capacity,state_id,state_name,network_id,network_name,network_background_img,network_logo_img,network_sponsors
0,2023-09-20 19:17:00,562,47.370238,8.514841,Goldbrunnenplatz,Schrennengasse 37,8003,Zürich,"[{'id': 855, 'name': '500052', 'ebike_battery_...",[],True,19,1,Active,6,Zürich,None,https://www.publibike.ch/static-content/Netz6.svg,[]
1,2023-09-20 19:17:00,185,46.946664,7.400752,Bahnhof Stöckacker,Untermattweg 36,3027,Bern,"[{'id': 3207, 'name': '102789', 'ebike_battery...",[],True,10,1,Active,5,Bern,None,https://www.publibike.ch/static-content/Netz5.svg,[]
2,2023-09-20 19:17:00,536,47.381591,8.490380,Freilager,Rautistrasse 60,8048,Zürich,"[{'id': 3927, 'name': '103301', 'ebike_battery...",[],True,26,1,Active,6,Zürich,None,https://www.publibike.ch/static-content/Netz6.svg,[]
3,2023-09-20 19:17:00,470,46.943817,7.395582,Schwabstrasse Kreisel,Keltenstrasse 41,3018,Bern,"[{'id': 3358, 'name': '501280', 'ebike_battery...",[],True,32,1,Active,5,Bern,None,https://www.publibike.ch/static-content/Netz5.svg,[]
4,2023-09-20 19:17:00,883,46.960798,7.433964,Schwimmhalle Neufeld,Neubrückstrasse 133,3012,Bern,"[{'id': 4972, 'name': '502576', 'ebike_battery...",[],True,80,1,Active,5,Bern,None,https://www.publibike.ch/static-content/Netz5.svg,[]


In [245]:
bike_df_list = []
sponsor_df_list = []

for station_id, vehicle_info, sponsor_info in zip(
    stations["id"], stations["vehicles"], stations["sponsors"]
):
    # Expand the bike information into separate df to large back onto the main df later
    bike_df = pd.json_normalize(vehicle_info, sep="_").assign(station_id=station_id)
    bike_df_list.append(bike_df)

    # same for the sponsors
    sponsor_df = pd.json_normalize(sponsor_info, sep="_").assign(station_id=station_id)
    sponsor_df_list.append(sponsor_df)

In [246]:
bikes = pd.concat(bike_df_list).rename(columns={"id": "bike_id", "name": "bike_name"})
bikes.head()

C:\Users\mathi\AppData\Local\Temp\ipykernel_20916\3834722591.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


,bike_id,bike_name,ebike_battery_level,type_id,type_name,station_id
0,855.0,500052,34.0,2.0,E-Bike,562
1,648.0,101391,NaN,1.0,Bike,562
2,1608.0,501844,13.0,2.0,E-Bike,562
0,3207.0,102789,NaN,1.0,Bike,185
1,4933.0,502981,80.0,2.0,E-Bike,185


In [247]:
sponsors = pd.concat(sponsor_df_list).rename(columns={"id": "sponsor_id", "name": "sponsor_name"})
sponsors

,station_id,sponsor_id,sponsor_name,image,url
0,809,47.0,Sponsor 47,https://www.publibike.ch/static-content/Sponso...,None
0,793,26.0,Sponsor 26,https://www.publibike.ch/static-content/Sponso...,None
0,8,5.0,Sponsor 05,https://www.publibike.ch/static-content/Sponso...,None
0,626,28.0,Sponsor 28,https://www.publibike.ch/static-content/Sponso...,None
0,18,14.0,Sponsor 14,https://www.publibike.ch/static-content/Sponso...,None
...,...,...,...,...,...
0,769,26.0,Sponsor 26,https://www.publibike.ch/static-content/Sponso...,None
0,233,26.0,Sponsor 26,https://www.publibike.ch/static-content/Sponso...,None
0,640,26.0,Sponsor 26,https://www.publibike.ch/static-content/Sponso...,None
0,644,26.0,Sponsor 26,https://www.publibike.ch/static-content/Sponso...,None


Let's join these dataframes back together for final analysis:

In [248]:
df = (
    stations.merge(bikes, how="left", left_on="id", right_on="station_id")
    .drop("station_id", axis=1)
    .merge(sponsors, how="left", left_on="id", right_on="station_id")
    .drop(
        [
            "station_id",
            "vehicles",
            "sponsors",
            "network_background_img",
            "network_logo_img",
            "network_sponsors",
            "image",
            "url",
        ],
        axis=1,
    )
    .rename(columns={"id": "station_id"})
)
df.head()

,timestamp,station_id,latitude,longitude,name,address,zip,city,is_virtual_station,capacity,state_id,state_name,network_id,network_name,bike_id,bike_name,ebike_battery_level,type_id,type_name,sponsor_id,sponsor_name
0,2023-09-20 19:17:00,562,47.370238,8.514841,Goldbrunnenplatz,Schrennengasse 37,8003,Zürich,True,19,1,Active,6,Zürich,855.0,500052,34.0,2.0,E-Bike,NaN,NaN
1,2023-09-20 19:17:00,562,47.370238,8.514841,Goldbrunnenplatz,Schrennengasse 37,8003,Zürich,True,19,1,Active,6,Zürich,648.0,101391,NaN,1.0,Bike,NaN,NaN
2,2023-09-20 19:17:00,562,47.370238,8.514841,Goldbrunnenplatz,Schrennengasse 37,8003,Zürich,True,19,1,Active,6,Zürich,1608.0,501844,13.0,2.0,E-Bike,NaN,NaN
3,2023-09-20 19:17:00,185,46.946664,7.400752,Bahnhof Stöckacker,Untermattweg 36,3027,Bern,True,10,1,Active,5,Bern,3207.0,102789,NaN,1.0,Bike,NaN,NaN
4,2023-09-20 19:17:00,185,46.946664,7.400752,Bahnhof Stöckacker,Untermattweg 36,3027,Bern,True,10,1,Active,5,Bern,4933.0,502981,80.0,2.0,E-Bike,NaN,NaN
